## Asignación de Recursos Financieros Productores de Caña MX

### Descripción del Problema:
Dadas las *dificultades de productividad* a la que se enfrentan los Productores de Caña en México, debidos principalmente a condiciones climáticas atípicas y escasez de agua, se hace necesaria la asignación de recursos financieros para garantizar la producción de caña que permita asegurar la operación de los ingenios del grupo.
Esta asignación se plantea como un problema de optimización de recursos financieros (propios o de entidades) que permita asegurar la producción de caña necesaria, sujeta a un presupuesto finito y a la capacidad de re-pago de los productores de caña.

Se formula un ***Problema de Optimización Lineal*** con los siguientes parámetros:

- **Elegibilidad (0-1)**: Este parámetro indicará si este productor de caña *califica* para un préstamo basado en distintos criterios, como productividad mínima, monto máximo a financiar, interés en seguir produciendo caña, etc.
- **Beneficio (USD)**: Este parámetro estima el beneficio que PSA (o las entidades financieras) obtiene(n) al final de horizonte temporal por financiar a un productor de caña específico, que incluye tanto el re-pago del monto prestado más intereses, como también los beneficios estimados relacionados a la productividad esperada de sus terrenos destinados a siembra de caña.
- **Monto a Financiar (USD)**: Magnitud de la ayuda financiera que el productor necesita.
- **Área Efectiva (ha)**: Se refiere al área efectiva de caña que se proyecta para sus terrenos destinados a siembra de caña.
- **Rendimiento de Caña (t Caña/ ha)**: Parámetro de productividad de caña por unidad de área efectiva.
- **Presupuesto (USD)**: Monto total anual de ayuda a asignar, durante el horizonte de tiempo.
- **Total de Caña (t Caña)**: Cantidad total de caña anual requerida de los productores en necesidad de ayuda financiera.
- **Riesgo de Impago (%)**: Probabilidad de un productor de caña de no cumplir con los compromisos de pago.

### Formulación del Problema de Optimización

#### Conjuntos y Notaciones:

Sean:
- $I$ el conjunto de Productores de Caña, con $i$ el i-ésimo productor de caña, $i \in \{1,2,...,n\}$.
- $J$ el conjunto de Años en el Horizonte de Análisis, con $j$ el j-ésimo año en el horizonte, $j \in \{1,2,...,m\}$.

#### Parámetros:
- $E_{i} \in \{0,1\}, \forall i \in \{1,2,...,n\} $ la Elegibilidad a Financiamiento del i-ésimo Productor de Caña.
- $B_{i,j} \in \mathbb{R}, \forall i \in \{1,2,...,n\}, \forall j \in \{1,2,...,m\}$ el Beneficio por Financiar al Productor $i$ en el año $j$.
- $F_{i} \in \mathbb{R}, \forall i \in \{1,2,...,n\}$ el Financiamiento Requerido por el i-ésimo Productor de Caña.
- $A_{i,j} \in \mathbb{R}, \forall i \in \{1,2,...,n\}, \forall j \in \{1,2,...,m\}$ el Área Efectiva de Caña del Productor $i$ en el año $j$.
- $C_{i,j} \in \mathbb{R}, \forall i \in \{1,2,...,n\}, \forall j \in \{1,2,...,m\}$ el Rendimiento Ponderado de Caña (TCH) del Productor $i$ en el año $j$.
- $R_{i} \in [0,1], \forall i \in \{1,2,...,n\}$

#### Variables de Decisión:
- $X_i \in \{0,1\}, \forall i \in \{1,2,...,n\}$ la variable que indica la decisión de financiar al i-ésimo Productor de Caña en todo el horizonte de análisis.

#### Función Objetivo Beneficio Financiación Productores MX

- $\max\limits_{\forall i \in I, \forall j \in J} \sum_{i=1}^{n}\sum_{j=1}^{m} X_{i}*E_{i}*B_{ij} - \sum_{i=1}^{n} R_{i}*X_{i}$

#### Restricciones

Presupuesto Anual para Financiamiento:

- $\sum_{i=1}^{n} X_{i}*F_{i,j} \leq Budget_{j}; \forall j \in J$

Demanda Anual de Caña:

- $\sum_{i=1}^{n} X_i*A_{i,j}*C_{i,j} + Caña_{Propia} + Caña_{Productores Seguros} \geq Caña_{j};  \forall j \in J$

No-Negatividad del Beneficio:

- $\sum_{i=1}^{n}\sum_{j=1}^{m} X_{i}*E_{i}*B_{ij} \geq 0$

Tolerancia Máxima al Riesgo de Incumplimiento (Probabilidad de Impago):

- $R_{i} \leq R_{MAX}, \forall i \in \{1,2,...,n\}$

Monto de Financiamiento Máximo a Productor Individual:

- $F_{i} \leq F_{MAX}, \forall i \in \{1,2,...,n\}$

Variables de Decisión:

- $X_i \in \{0,1\}, \forall i \in \{1,2,...,n\}$

In [1]:
# librerías de optimización

import gurobipy as grb
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'ProductoresMX_3Years.csv')
df

,ID,Elegibilidad,Probabilidad_Impago,Beneficio_Year1,Financiamiento_Year1,Area_Year1,Rendimiento_Year1,Beneficio_Year2,Financiamiento_Year2,Area_Year2,Rendimiento_Year2,Beneficio_Year3,Financiamiento_Year3,Area_Year3,Rendimiento_Year3
0,1,0,0.593092,401.084772,69282.256320,31.647425,91.376103,36168.99325,32167.610590,44.186404,111.185099,28676.94343,40185.681250,68.374044,76.266904
1,2,1,0.243090,13564.632850,42239.396880,47.291438,65.648293,28645.12687,48809.416540,31.883765,114.210422,26224.11852,6445.835788,41.413101,96.013109
2,3,1,0.310311,5265.660355,5652.194208,29.812790,92.229906,30059.41393,2383.222947,37.818307,122.182482,31682.94579,23664.629800,57.644285,40.867040
3,4,0,0.061366,16616.681560,24553.164720,54.831072,83.661621,26695.44489,16561.069480,40.817347,79.575744,27374.75550,17165.263050,44.620963,103.229436
4,5,0,0.409707,60761.513750,41358.659810,45.141183,68.472908,30331.89355,25359.982050,46.068943,123.935754,33158.37066,26285.169530,22.307319,100.255832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,0,0.112297,25882.669890,8362.916731,38.048945,88.454536,19018.96427,66364.697490,43.059326,93.240765,30541.25843,7350.147264,24.946519,117.056962
296,297,1,0.217216,26214.931490,19929.306000,34.621520,71.000991,24714.52758,39360.416870,36.519768,135.421036,30244.08820,50841.911010,51.658685,126.577336
297,298,0,0.870100,-4283.711224,39160.399500,33.747162,110.683187,33219.79570,14127.757810,31.157131,97.464444,30019.68793,67049.439760,34.215814,63.807104
298,299,0,0.772284,53693.599340,46298.732540,28.998073,132.687710,32722.54808,58456.392450,34.642015,122.747747,29350.92596,54528.148180,24.784573,100.462497


#### Parámetros de Optimización

In [3]:
## productores
Producers = np.array(df)

# número de productores
n = len(df['ID'])

# número de años
a = 3

# lista de productores
I = {i for i in range(1,n+1)}

# lista de años
J = {j for j in range(1,a+1)}

# parámetros
BUDGET = [1000000,800000,900000]
CANA = [50000,70000,80000]
RISK = 0.15
FUNDING = 50000

E = {i:Producers[i-1,1] for i in I}
R = {i:Producers[i-1,2] for i in I}
B = {(i,j):Producers[i-1,4*(j-1)+3] for i in I for j in J}
F = {(i,j):Producers[i-1,4*(j-1)+4] for i in I for j in J}
A = {(i,j):Producers[i-1,4*(j-1)+5] for i in I for j in J}
C = {(i,j):Producers[i-1,4*(j-1)+6] for i in I for j in J}

#### Modelo de Optimización

In [4]:
# modelo de optimización
model = grb.Model("Seleccion de Productores")

# variables de decisión
x = model.addVars(I, vtype = grb.GRB.BINARY)

Set parameter Username


#### Función Objetivo

In [5]:
# función objetivo
model.setObjective(grb.quicksum(E[i]*B[(i,j)]*x[i] - R[i]*x[i] for i in I for j in J), grb.GRB.MAXIMIZE)

#### Restricciones

In [6]:
# restricciones
model.addConstrs((grb.quicksum(F[(i,j)]*x[i] for i in I) <= BUDGET[j-1] for j in J), name = "Financiamiento Anual Maximo")
model.addConstrs((grb.quicksum(C[(i,j)]*A[(i,j)]*x[i] for i in I) >= CANA[j-1] for j in J), name = "Demanda Anual de Caña")
model.addConstr((grb.quicksum(E[i]*B[(i,j)]*x[i] for i in I for j in J) >= 0), name = "No-Negatividad del Beneficio Financiero")
model.addConstrs((R[i]*x[i] <= RISK for i in I), name = "Tolerancia al Riesgo de Incumplimiento")
model.addConstrs((F[(i,j)]*x[i] <= FUNDING for i in I for j in J), name = "Financiamiento Individual Maximo")

model.update()

### Solución al Problema de Optimización de Programación Entera-Mixta (MIP)

In [7]:
# optimización
model.optimize()

if model.status != grb.GRB.status.OPTIMAL:
    print("No se encontró una solución factible...")
else:
    print("El valor optimo de la función objetivo es %g"%model.objVal)
    resultado_objetivo = model.objVal
    resultado_seleccion_productores = [i for i in I if x[i].X >= 1]

    df_salida = pd.DataFrame({"valor función objetivo": [resultado_objetivo],
                              "productores seleccionados": [resultado_seleccion_productores]})

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1207 rows, 300 columns and 3162 nonzeros
Model fingerprint: 0x125deb15
Variable types: 0 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+05]
  Objective range  [3e-02, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+06]
Presolve removed 1206 rows and 289 columns
Presolve time: 0.00s
Presolved: 1 rows, 11 columns, 11 nonzeros
Variable types: 0 continuous, 11 integer (11 binary)
Found heuristic solution: objective 917485.40148

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 917485 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.174854014809e+05, best bound 9.174877913912e+05, gap 0.0003%
El valor optimo de l

In [8]:
df_salida

,valor función objetivo,productores seleccionados
0,917485.401481,"[4, 63, 85, 86, 98, 101, 106, 127, 148, 151, 1..."


#### Análisis de Resultados

In [9]:
productores = df_salida['productores seleccionados'][0]
df_final = df.loc[df['ID'].isin(productores)].reset_index()
df_final.drop('index', axis=1, inplace=True)
df_final.head()

,ID,Elegibilidad,Probabilidad_Impago,Beneficio_Year1,Financiamiento_Year1,Area_Year1,Rendimiento_Year1,Beneficio_Year2,Financiamiento_Year2,Area_Year2,Rendimiento_Year2,Beneficio_Year3,Financiamiento_Year3,Area_Year3,Rendimiento_Year3
0,4,0,0.061366,16616.68156,24553.164720,54.831072,83.661621,26695.44489,16561.069480,40.817347,79.575744,27374.75550,17165.263050,44.620963,103.229436
1,63,1,0.132850,33358.08915,30674.369780,54.740492,112.488847,33307.66118,32168.008090,55.370221,117.930150,30772.21113,8046.305471,51.873806,123.472680
2,85,0,0.009906,29470.97782,30157.595260,64.685665,77.997509,37317.03288,725.545304,44.314604,97.247723,30752.87091,30228.793400,51.305439,108.989374
3,86,1,0.046306,-4730.78006,1015.000207,46.693940,56.482049,28256.93567,42742.458180,20.927875,113.977476,22964.73075,19451.206510,36.703146,96.693506
4,98,1,0.085273,22447.10137,28110.186780,29.120179,77.598183,32370.66462,8025.022782,39.682175,123.593206,31085.64389,32819.947570,24.006978,88.722667


In [10]:
print("============================== RESTRICCIONES =============================")
print("")
print("Dados los siguientes parámetros:")
print("Presupuesto Máximo Anual para Financiar (USD):",BUDGET)
print("Monto Máximo Individual de Financimiento por Productor (USD):",FUNDING)
print("Demanda Mínima Anual de Caña (t):",CANA)
print("Probabilidad Máxima de Impago tolerada (%):",RISK*100)
print("")
print("=========================== RESUMEN RESULTADOS ===========================")
print("")
print("Se han seleccionado: %0.2f de %0.2f productores de caña" % (len(df_final['ID']), n))
print("El beneficio estimado total en %0.2f años es de USD %0.2f" % (a,sum(df_final['Beneficio_Year1'])+sum(df_final['Beneficio_Year2'])+sum(df_final['Beneficio_Year3'])))
print("Los beneficios estimados por año son (USD):", [np.round(sum(df_final['Beneficio_Year1'],2)),np.round(sum(df_final['Beneficio_Year2'],2)),np.round(sum(df_final['Beneficio_Year3'],2))] )
print("El financiamiento total en %0.2f años es de USD: %0.2f" % (a,sum(df_final['Financiamiento_Year1'])+sum(df_final['Financiamiento_Year2'])+sum(df_final['Financiamiento_Year3'])))
print("Los financiamientos requeridos por año son (USD):", [np.round(sum(df_final['Financiamiento_Year1'],2)),np.round(sum(df_final['Financiamiento_Year2'],2)),np.round(sum(df_final['Financiamiento_Year3'],2))] )
print("El suministro de caña total en %0.2f años es de %0.2f toneladas" % (a,sum(df_final['Area_Year1']*df_final['Rendimiento_Year1'])+sum(df_final['Area_Year2']*df_final['Rendimiento_Year2'])+sum(df_final['Area_Year3']*df_final['Rendimiento_Year3'])))
print("El suministro de caña por año es (t):", np.round([sum(df_final['Area_Year1']*df_final['Rendimiento_Year1']),sum(df_final['Area_Year2']*df_final['Rendimiento_Year2']),sum(df_final['Area_Year3']*df_final['Rendimiento_Year3'])],2))
print("El riesgo máximo de impago es del %0.2f porciento" % (np.max(df_final['Probabilidad_Impago'])*100))
print("El riesgo medio de impago es del %0.2f porciento" % (np.mean(df_final['Probabilidad_Impago'])*100))
print("El riesgo mínimo de impago es del %0.2f porciento" % (np.min(df_final['Probabilidad_Impago'])*100))
print("")
print("Los productores seleccionados son:",productores)
print("")
print("==========================================================================")


============================== RESTRICCIONES =============================

Dados los siguientes parámetros:
Presupuesto Máximo Anual para Financiar (USD): [1000000, 800000, 900000]
Monto Máximo Individual de Financimiento por Productor (USD): 50000
Demanda Mínima Anual de Caña (t): [50000, 70000, 80000]
Probabilidad Máxima de Impago tolerada (%): 15.0

=========================== RESUMEN RESULTADOS ===========================

Se han seleccionado: 20.00 de 300.00 productores de caña
El beneficio estimado total en 3.00 años es de USD 1796316.27
Los beneficios estimados por año son (USD): [572183.0, 628049.0, 596091.0]
El financiamiento total en 3.00 años es de USD: 1366259.32
Los financiamientos requeridos por año son (USD): [441750.0, 426474.0, 498042.0]
El suministro de caña total en 3.00 años es de 241439.25 toneladas
El suministro de caña por año es (t): [81576.77 79858.21 80004.27]
El riesgo máximo de impago es del 13.28 porciento
El riesgo medio de impago es del 8.94 porciento
El